In [16]:
import pandas as pd
import numpy as np
import textatistic as tx
import nltk
import os
import re

from threading import Thread
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


In [5]:
#   DATA CLEANING

neg_data = []
pos_data = []

dir_neg = r"/home/kinged/Documents/Random/NLP_init/test/neg" #"C:\Users\KingED\Documents\Random\NLP_init\test\neg" for windows
dir_pos = r"/home/kinged/Documents/Random/NLP_init/test/neg" #"C:\Users\KingED\Documents\Random\NLP_init\test\pos" for windows

def maker(loc: list, path: str):
    for files in os.listdir(path):
        os.chdir(path)
        with open(files, 'rb+') as file:
            loc.append(file.readlines())
       
maker(neg_data, dir_neg)
maker(pos_data, dir_pos)


pattern_1 = r"^b\"|^b\'"
pattern_2 = r"<br\s\/\>"
patter_3 = r'\\|\"|\''

pos_word = []
neg_word = []

#FIXME : convert these loops into functions
for c in pos_data:
    word = str(c[0])
    word = re.sub(pattern_2, "", word)
    word = re.sub(pattern_1, "", word)
    word = re.sub(patter_3, "", word)
    pos_word.append(word)
    
for c in neg_data:
    word = str(c[0])
    word = re.sub(pattern_2, "", word)
    word = re.sub(pattern_1, "", word)
    word = re.sub(patter_3, "", word)
    neg_word.append(word)

 

In [6]:
# FIXME: make this function compute the gunningfog and flesch scores at the same time 

fog_scores = []
flesch_score = []
issues = []
for x in pos_word:
    try:
        fog = tx.gunningfog_score(x)
        flesch = tx.flesch_score(x)
        fog_scores.append(fog)
        flesch_score.append(flesch)
      
    except ZeroDivisionError:
        issues.append(x) 


In [7]:
average_fog = sum(fog_scores)/ len(fog_scores)
average_flesch = sum(flesch_score)/ len(flesch_score)

print(f"the average flesch score for poistive reviews is {average_flesch}")
print(f"the average runningfog scores for the postive reviews is {average_fog}")

the average flesch score for poistive reviews is 80.61120107848635
the average runningfog scores for the postive reviews is 9.199174109143955


In [8]:
doc_pos = pd.DataFrame({'review': pos_word, 'score': 1})
doc_neg = pd.DataFrame({'review': neg_word, 'score': 0})

data  = pd.concat([doc_neg, doc_pos], ignore_index=True)
data

,review,score
0,the usual disclaimer - I do not give 1 star ra...,0
1,"The Tender Hook, or, Who Killed The Australian...",0
2,Go Fish garnered Rose Troche rightly or wrongl...,0
3,"There are movies that are leaders, and movies ...",0
4,Do the following: Get a copy of this movie and...,0
...,...,...
24995,"I like CKY and Viva La Bam, so I couldnt resis...",1
24996,"I love the Jurassic Park movies, they are thre...",1
24997,The movie is not as funny as the directors pre...,1
24998,I watched the version with pathetic American o...,1


In [13]:
os.chdir(r"/home/kinged/Documents/Random/NLP_init")

for file in os.listdir():
    if file != 'review.csv':
        data.to_csv('review.csv')
    else:
        print("file aready exists")

file aready exists


In [14]:
data_array = data.values
data_array[0, :]

array(['the usual disclaimer - I do not give 1 star ratings to movies which are harmless, bad, low budget and silly, although they may deserve it. These films are often funny, and get rated 2-4 based sheerly on entertainment value - not as a representation of their exemplary film artistry. This film fits this model perfectly. It is a Mexican monster movie, riddled with voice-over narrative and extremely weak not-so-special effects. The makeup is not that bad, and the acting is sometimes quite entertaining, but this film is almost as silly as Aliens vs Predator and the script isnt half as slick (Aliens vs Predator might get a 1 from me, but I want to see it again before I commit).The plot is ridiculous, but deliciously convoluted. If youve read this far, you must really want to know... A group of remarkably unscientific scientists comprise the main characters. Most of them are heroes - sort of - but one is (of course) mad, and quite perverse. This mad scientist invents a laughable nucle

In [15]:
"""
TODO: TOKENIZATION
TODO: REMOVE STOPWORDS
TODO: LEMMATIZE
TODO: WORD AND CHARACTER COUNT

"""
nltk.download("punkt")

def token_maker(word: list) -> list:
    words = []
    for x in range(len(pos_word)):
        words.append(word_tokenize(word[x]))
    return words

pos_tokens = token_maker(pos_word)
neg_tokens = token_maker(neg_word)



LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/kinged/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [ ]:
# SPACEY
"""
TODO: NER
TODO: POS TAGGING
"""

# nlp = spacy.load("en_core_web_sm")
# doc = nlp(doc)

# def ner_pos():
#     ners = [(token.text, token.label_) for word in doc]
#     tags = [(token.text, token.tags_) for word in doc]
    
word = word_tokenize(pos_word[1])

lemma = WordNetLemmatizer()
tfid = TfidfVectorizer()

# lemma.lemmatize(word)
tfidword = tfid.fit_transform(data['review'])

In [ ]:
counter = CountVectorizer(lowercase=True,
                          strip_accents='ascii',
                          tokenizer=None,
                          stop_words=list(STOP_WORDS),
                          )

encoder = LabelEncoder()
data['score'] = encoder.fit_transform(data['score'])

x = data['review'].values
y = data['score'].values
xtrain, xtest, ytrain, ytest = train_test_split(x, y, shuffle=True, test_size=0.3, random_state=34)


new_xtrain = counter.fit_transform(xtrain)
new_xtest = counter.transform(xtest)

print(new_xtrain.shape)

model = MultinomialNB(alpha=1)
model.fit(new_xtrain, ytrain)
pred = model.predict(new_xtest)
print(f"the accuracy of the model is: {accuracy_score(pred, ytest) * 100}")

c:\Users\KingED\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 'nt', 've'] not in stop_words.
  warnings.warn(


(17500, 68662)
the accuracy of the model is: 87.14666666666666
